# Motivation

论文原文：[《Entire Space Multi-Task Model: An Effective Approach for
Estimating Post-Click Conversion Rate》](https://arxiv.org/abs/1804.07931)


CVR预估相较于CTR预估有两个关键问题：
* **Sample Selection Bias（SSB）** 转化是在点击之后才“有可能”发生的动作，传统CVR模型通常以点击数据为训练集，其中点击未转化为负例，点击并转化为正例。但是训练好的模型实际使用时，则是对整个空间的样本进行预估，而非只对点击样本进行预估。即是说，训练数据与实际要预测的数据来自不同分布，这个偏差对模型的泛化能力构成了很大挑战。
* **Data Sparsity（DS）** 作为CVR训练数据的点击样本远小于CTR预估训练使用的曝光样本。

一些策略可以缓解这两个问题，e.g.：
* 从曝光集（IMP）中对未点击样本抽样作为负例（也即无标采样），缓解SSB
* 对转化样本过采样缓解DS
但两个方案都未实质上解决上面任一个问题。

可以看到：点击->转化，本身是两个强相关的连续行为，在模型结构中显式的考虑这种“行为链”关系，从而可以在整个空间上进行训练及预测。涉及CTR与CVR两个任务，也即多任务学习（MTL，Multi-Task Learning）

在整个行为链上，前提假设仍是各单步骤的概率预测独立。

$$
\begin{align}
\underbrace{p(z\&y=1|x)}_{pCTCVR}=\underbrace{p(z=1|y=1,x)}_{pCVR}\underbrace{p(y=1|x)}_{pCTR}
\end{align}
$$

其中$z$和$y$分别代表转化和点击。在全部样本空间中，CTR对应的label为点击，CTCVR对应的label为点击且转化。**<font color="red">这两个任务可以使用全部样本，通过学习这两个任务，再根据上式隐式地学习CVR任务</font>**。

 <img style="display: block; margin: 0 auto;" src="../../../assets/images/esmm.png" width = "500" height = "600" alt="ESMM网络结构" align=center />

 Tips：
 * 共享Embedding：CTR和CVR使用相同的特征和特征embedding，两个MLP都是从concatenate之后才学习各部分独享的参数；
 * 隐式pCVR：这里pCVR（粉丝节点）尽是网络中的一个variable，并没有显式的监督信号（也即没有对应的label，仅CTR和CTCVR有对应的ground truth），对应的损失函数为

$$
\begin{align}
\mathbf{L}(\theta_{cvr},\theta_{ctr})=\Sigma_{i=1}^Nl(y_i,f(x_i;\theta_{ctr}))+\Sigma_{i=1}^Nl(y_i\&z_i,f(x_i;\theta_{ctr}) * f(x_i;\theta_{cvr}))
\end{align}
$$


## 一些思考

* **<font color="red">CVR模型预测概率会被高估</font>**：CVR和CTR的训练预测虽然都是在全量的曝光集（IMP）上训练预测，但要注意CVR的一个前提假设是如果全部IMP都被点击，然后定义转换的概率（而CTCVR是IMP被点击且转换的概率），也即认为用户的点击和点击后的转换是独立。但这个假设通常不成立，因为往往被点击的更容易被转化，而那些IMP但为CLK的实际上被增加了一个先验假设。也即ESMM实际为有偏预测（已在论文[《Large-scale Causal Approaches to Debiasing Post-click Conversion Rate Estimation with Multi-task Learning》](https://arxiv.org/abs/1910.09337)中被证实）
* **推理和训练的数据集和理论不符**： ${pCVR}=p(z=1|y=1,x)$ 表示CVR概率，理论上应用CLK的样本（y=1，x），但实际上用的是IMP样本（x）。所以原公式应有推导错误。按照motivation，推导应如下：

$$
\begin{align}
\underbrace{p(z=1|x)}_{pCVR}=\underbrace{p(z=1,y=1|x)}_{pCTCVR}+ p(z=1,y=0|x)=p(z=1|y=1,x)\underbrace{p(y=1|x)}_{pCTR}+ p(z=1,y=0|x)
\end{align}
$$

从上式中得到两个结论：
* 真实的CVR应该是CTCVR加上未被点击但应该转换的部分，这个是符合常理的
* 原文中实际上认为：p(z=1|x)=p(z=1|y=1,x)，而这样的假设本质上是<font color="red">认为一个样本是否转换是和点击无关的</font>，（虽然不符合常识，但理论上是保持了一致性的）证明如下：

$$
\begin{align}
p(z=1|y=1,x) &= p(z=1|y=1,x)p(y=1|x)+p(z=1,y=0|x)\\
\frac{p(z=1,y=1|x)}{p(y=1|x)}p(y=0|x) &= p(z=1,y=0|x)\\
\frac{p(z=1,y=1|x)}{p(z=1,y=0|x)} &= \frac{p(y=1|x)}{p(y=0|x)}\\
\frac{p(z=1,y=1|x)}{p(z=1,y=0|x)} &= \frac{p(z=1,y=1|x)+p(z=0,y=1|x)}{p(z=1,y=0|x)+p(z=0,y=0|x)}\\
\frac{p(z=1,y=1|x)}{p(z=0,y=1|x)} &= \frac{p(z=1,y=0|x)}{p(z=0,y=0|x)}\\
\end{align}
$$

对于二元随机变量z和y，当且仅当z与y独立时上式成立，所以

$$
\begin{align}
\underbrace{p(z\&y=1|x)}_{pCTCVR}=\underbrace{p(z=1|y=1,x)}_{pCVR}\underbrace{p(y=1|x)}_{pCTR}=\underbrace{p(z=1|x)}_{pCVR}\underbrace{p(y=1|x)}_{pCTR}
\end{align}
$$

从因果的角度上，上式时存在歧义的，无CLK的数据如何能够转化呢？